
# Cargar los datos y realizar un análisis exploratorio y una evaluación de la calidad de los datos necesarios para el resto del caso.
## Evaluar la integridad, validez y actualidad de los datos y proponer estrategias de mitigación de los posibles problemas encontrados.

In [ ]:
# Importamos las librerias
import numpy as np
import pandas as pd
import datetime

In [ ]:
# Cargamos el dataset desde un fichero local. Nota: Antes de su carga, se examinó el archivo y se identificaron, columnas cuyos datos eran fechas. 
# Estas columnas las importamos como tipo datetime usando el argumento parse_dates. 
# Las columnas c_jail_in & c_jail_out, al llevar un formato distinto, las pasaremos a formato datetime a posteriori.
compas_raw=pd.read_csv('../input/compas-scores/compas-scores.csv', parse_dates=['compas_screening_date', 'dob', 'c_offense_date', 'c_arrest_date', 'r_offense_date', 'r_jail_in', 'r_jail_out', 'vr_offense_date'])
compas_raw

In [ ]:
compas_raw['c_jail_in']= pd.to_datetime(compas_raw['c_jail_in'])
compas_raw['c_jail_out']= pd.to_datetime(compas_raw['c_jail_out'])

In [ ]:
# Vemos de la funcion .info que no disponemos de ninguna columna de tipo "category" y dado que hay campos como "sexo" o "raza", este tipo de objeto es más conveniente.
# Procederemos a identificar que columnas deberiamos transformar en tipo "category".
compas_raw.nunique()

In [ ]:
# Identificamos las columnas con un bajo número de valores únicos y comprobamos sus valores únicos.
for col in ['sex', 'age_cat', 'race', 'c_charge_degree', 'is_recid', 'r_charge_degree', 'is_violent_recid', 'v_score_text', 'score_text']:
    print(compas_raw[col].unique())

In [ ]:
for col in ['sex', 'age_cat', 'race', 'c_charge_degree', 'is_recid', 'r_charge_degree', 'is_violent_recid', 'v_score_text', 'score_text']:
    compas_raw[col] = compas_raw[col].astype('category')

In [ ]:
# Ahora que las columnas estan guardadas en los formatos más pertinentes para su estudio, 
compas_raw.describe(include = 'category')

In [ ]:
compas_raw.describe(include = 'object')

In [ ]:
compas_raw.describe(include = np.number)

In [ ]:
# Tras hacer esta exploración inicial, a continuación se empezará a explorar las dimensiones de la calidad de los datos.

# Vemos que hay nombres que se repiten, por lo que necesitamos saber si hay un error o si son personas distintas. 
compas_raw[compas_raw.name.duplicated(keep = False)]

In [ ]:
#Viendo que hay 328 nombres (incl. nombre y apellido) comprobamos el más recurrente prar verificar si es la misma persona.
compas_raw.loc[compas_raw['name'] == 'michael cunningham']

In [ ]:
# Se puede concluir de la comprobación anterior que las personas que comparten nombre no son la misma persona. Esto tiene sentido ya que no se haria un registro nuevo en COMPAS sino
# que se consideraria el evento una reincidencia.

### Investigación de la actualidad de los datos.

In [ ]:
# Para tratar de entender un poco mejor el dataframe, identificaremos las filas con valores nulos
compas_raw.isnull().sum()

In [ ]:
''' Vemos que existen campos nulos/vacios en los días anteriores al arresto inicial. Esta cantidad coincide con la cantidad de las fechas de ingreso en prisión y salida.
Es posible que esto se deba a que hay personas que no han cometido un crimen desde el inicio del sistema COMPAS.
'''
comprension = compas_raw[['compas_screening_date', 'juv_fel_count', 'priors_count', 'c_jail_in', 'is_recid', 'r_offense_date', 'is_violent_recid', 'vr_offense_date']]
comprension.head()

In [ ]:
'''En esta exploración algo llamativo fue encontrar 3 valores únicos del campo "is_recid".
Tiene sentido que este campo sea binario. Es decir, debería indicar si reincidió o no. 
Esto puede deberse a un problema con la integridad de los datos, pero esto se explorará más adelante.'''

# Se procede a continuación con la exploración de la actualidad de los datos. 

# Como se describe en el enunciado no sabemos a partir de que fecha ya no se recogen datos de reincidencias, arrestos iniciales etc.
# Tampoco se sabe cuando se empezó a usar el sistema COMPAS.
print("Primeras fechas:")
print(comprension[['compas_screening_date', 'c_jail_in', 'r_offense_date', 'vr_offense_date']].min())
print("\n")
print("Ultimas fechas:")
print(comprension[['compas_screening_date', 'c_jail_in', 'r_offense_date', 'vr_offense_date']].max())

De esta comprobación se puede ver que se empezaron a registrar casos en el sistema COMPAS el 01 de Enero de 2013. Por otro lado, los últimos datos registrados y que tenemos a nuestra disposición son del 29 de Marzo de 2016.Teniendo esto en cuenta, sabemos que no disponemos de datos de reincidiencias posteriores a esta última fecha.

Por último, viendo la primera y última fecha del compas_screening_date, podemos concluir que estos datos son los casos registrados en el sistema compas de los años 2013 y 2014, y es por esta razón que aúnque este dataset se ha alimentado hasta el 29 de Marzo de 2016, no hay nuevos casos registrados en el sistema más allá del año 2014.

De estos datos sabemos que la fecha de descarga de estos datos es posterior al 29 de Marzo de 2016.

### Investigación de la integridad de los datos.

La primera observación que se puede hacer es que desconocemos el ámbito geográfico o las poblaciones objetos de este estudio, haciendo que cualquier observación o predicción que se haga será afectada por los sesgos que puede presentar este dataset ya que no se tendrán en cuenta muchos de los detalles que pueden influir en los datos.

De todas maneras, se ha visto que el campo "is_recid" dispone de 3 valores únicos. Esta situación se comprueba a continuación. 

In [ ]:
comprension['is_recid'].unique()

In [ ]:
negativeR= comprension.loc[comprension['is_recid'] == -1]
negativeR

In [ ]:
comprension.loc[comprension['is_recid'] == 0]

In [ ]:
comprension.loc[comprension['is_recid'] == 1]

In [ ]:
# Antes de establecer nuestas conclusiones hacemos una útlima comprobación de los casos cuyo valor de "is_recid" es -1 .
for col in ['juv_fel_count', 'priors_count', 'c_jail_in', 'r_offense_date', 'is_violent_recid', 'vr_offense_date']:
    print(negativeR[col].unique())

In [ ]:
# Se comprueba el caso no nulo de la columna "c_jail_in"...
negativeR[negativeR['c_jail_in'].notnull()]

In [ ]:
compas_raw.loc[compas_raw['id']==7428]

Tras aislar el dataframe según los tres valores de la columna "is_recid" se puede concluir que el valor 1 representa aquellos casos que sí reinicidieron, donde is_violent_recid indica si ha sido una reincidencia por un acto violento. Por otro lado, el valor 0 representa aquellos casos en los que no hubo reincidencia, al menos hasta la la fecha del 29 de Marzo de 2016. Por último, el valor -1 representa los casos que sí están en el sistema COMPAS pero sin haber cometido un crimen registrado. El caso anterior (ID 7428) es posible que sea un error. También es posible que simplemente no se disponen de los datos pertinentes del resto de los casos. Aún así, dado que aparitr de los datos aportados no se puede determinar, esto supone un problema en la integridad de los datos.

### Investigación sobre la validez de los datos

En cuanto a la validez de los datos, no se ha detectado ningún error o falta en cuanto a esta dimensión.

# ¿Son los campos “is_recid” e “is_violent_recid” en este conjunto de datos adecuados para evaluar la precisión de las estimaciones de riesgo generadas por el sistema COMPAS? Si no es así, definir y calcular una feature que sí lo sea.

In [ ]:
'''Dado que se ha establecido que aquellos registros cuyo valor "is_recid" = -1 suponen un problema de integridad de los datos, estos se van a eliminar del dataframe
para proceder con la evaluación de las variables'''
df=compas_raw[(compas_raw['is_recid'] !=-1)&(compas_raw['decile_score'] !=-1)]

Como se ha identificado durante la investigación sobre la actualidad de los datos, es muy posible que los datos que se tienen de las reincidencias no sea completa ya que un caso puede reincidir después del 29 de Marzo de 2016. Por esta razón, para poder evaluar los datos se tienen que homogeneizar los datos en un periodo de tiempo determinado.

In [ ]:
# Primero necesitamos determinar desde que fecha se debe de considerar el inicio del periodo.
df[['c_offense_date', 'c_arrest_date', 'c_charge_desc']]

In [ ]:
# Entendemos por estos datos que para tener una "fecha de inicio" debemos de combinar las dos columnas. 
# Nota: c_arrest_date se cumplimenta en el caso de no haber cargos imputados.
df2=df.copy()
df2['c_offense_date'] = df2['c_offense_date'].astype(str)
df2['c_arrest_date'] = df2['c_arrest_date'].astype(str)
df2['start_date']=(df2.c_offense_date+df2.c_arrest_date)
df2['start_date'] = df2['start_date'].map(lambda x: x.lstrip('NaT').rstrip('NaT'))
df2['start_date']=pd.to_datetime(df2.start_date)

In [ ]:
# También establecemos el periodo a considerarse mediante una columna auxiliar denominada "end_date". Esta la podremos fijar en función de días.
from datetime import timedelta
def set_timeframe (timeframe_in_days):
    df2['end_date']=df2['start_date']+timedelta(days=timeframe_in_days)
    print(df2[['start_date','end_date']])

In [ ]:
# Para determinar un periodo de tiempo a considerar, creamos una columna que incluirá el tiempo hasta la reincidencia.
df2['time_until_recid']=df2['r_offense_date']-df2['start_date']

In [ ]:
# Calculamos la media de dias que tardaron en reincidir para cada decile score.
for score in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    print('Tiempo Medio de reincidencia de decile_score=',score,(df2[(df2.time_until_recid!='NaT')&(df2.decile_score==score)]['time_until_recid']).mean())

In [ ]:
# Viendo que el tiempo medio excede en la mayoria de los casos excede los 300 días, escogeremos para nuestro intervalo de tiempo, 365 días (un año completo).
set_timeframe (timeframe_in_days=365)
# Nuestra función establece los valores de la columna end_date y nos devuelve una comprobación:

In [ ]:
# Se ve que la última fecha de reincidencia es posterior a la fecha tope a considerar: "end_date"
df2[['start_date','end_date', 'r_offense_date']].max()

In [ ]:
'''Se crea un dataframe alternativo en el que se ha aplicado un filtro de los datos para que solamente aparezcan aquellos casos que reincidieron durante 
el periodo o aquellos casos que no reinicdieron'''

timeframe_df=df2[(df2['r_offense_date']<df2['end_date'])|(df2['r_offense_date'].isnull())]

In [ ]:
# Se ve ahora que la última reincidencia registrada ya no es posterior a la fecha tope.
timeframe_df[['start_date','end_date', 'r_offense_date']].max()

In [ ]:
for score in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    print('el número de decile_score=',score,'es',len(timeframe_df[timeframe_df['decile_score'] == score]), 'de los que', len(timeframe_df[(timeframe_df.is_recid == 1) & (timeframe_df.decile_score == score)]),', un',(((len(timeframe_df[(timeframe_df.is_recid == 1) & (timeframe_df.decile_score == score)]))/(len(timeframe_df[timeframe_df['decile_score'] == score])))*100),'%  reincidieron en menos de un año.')

Tras hacer este análisis, se puede observar que las columnas is_recid e is_violent recid, por si solas no son adecuadas para evaluar la precisión de las estimaciones ya que no tienen en cuenta un tiempo homogeneizado. A continuación se crea una feature que sí contemple un tiempo homogeneizado.

In [ ]:
df2['recid_within_timeframe']=np.where(df2['r_offense_date']<df2['end_date'],'YES','NO')
df2['violent_recid_within_timeframe']=np.where(df2['vr_offense_date']<df2['end_date'],'YES','NO')
df2[['recid_within_timeframe', 'violent_recid_within_timeframe']]

# El umbral para establecer medidas preventivas de la reincidencia es de 7 en adelante.
# Dado este umbral, generar una tabla de contingencia, explicando qué caso se considera como “positivo” (y, por lo tanto, cuáles son los errores de tipo I y los errores de tipo II).

In [ ]:
# Para que el análisis sea correcto, se debe continuar usando los datos de reincidencia dentro de un periodo de tiempo específico.
df2['contingencia_recid']=np.where(df2['decile_score']>=7,'YES','NO')

In [ ]:
# Se comprueba que la columna creada asigna el valor 1 a todos aquellos casos cuyo decile score es >=7.
df2[['decile_score','contingencia_recid']]

In [ ]:
# Creamos la tabla de contingencia.
tabla_contingencia = pd.crosstab(df2.contingencia_recid, df2.recid_within_timeframe)
tabla_contingencia

En nuestro caso consideramos un caso "positivo" aquel en el que sí se ponen medidas de contingencia y sí reinciden en el periodo determinado (un total de 1031 casos). Por lo tanto los errores de tipo I (o falso positivo) son aquellos en los que contingencia_recid="YES" pero recid_within_timeframe="NO" (un total de 1930 casos). Los errores tipo II (o falso negativo)son aquellos en los que contingencia_recid="NO" pero recid_within_timeframe="YES" (un total de 1405 casos). De cara a temas de seguridad ciudadana, caso los errores tipo II son más serios.

# El sistema asigna, de media, evaluaciones de riesgo más altas a los hombres que a las mujeres, y a las personas de raza afroamericana que a las de raza caucásica. Sin embargo, también las tasas de reincidencia son más altas para esos colectivos, aunque no está claro que la asignación de riesgo sea “justa” o no.

## Mostrar estas diferencias mediante representaciones gráficas y utilizarlas para analizar si la asignación de evaluaciones es justa o no. 

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()

In [ ]:
# Se elimina la columna 'time_until_recid' ya que se encontró que generaba problemas a la hora de la creación de las tablas debido a su formato.
df3=df2.copy()
df3.drop('time_until_recid', inplace=True, axis=1)

In [ ]:
df_compare_race=df3.copy()
df_compare_race=df_compare_race[(df_compare_race['race']=='Caucasian')|(df_compare_race['race']=='African-American')]

In [ ]:
# Dado que el boxplot estandar de Altair no muestra la media sino la mediana, se ha tenido que crear el boxplot de manera manual.
decile_by_sex = alt.LayerChart(data=df3).transform_aggregate(
    min="min(decile_score)",
    max="max(decile_score)",
    mean="mean(decile_score)",
    q1="q1(decile_score)",
    median="median(decile_score)",
    q3="q3(decile_score)",
    groupby=['sex']
).encode(
    x='sex:N',
    tooltip=['min:Q', 'q1:Q', 'median:Q','mean:Q', 'q3:Q', 'max:Q']
).add_layers(
    alt.Chart().mark_rule().encode(y='min:Q', y2='max:Q'),
    alt.Chart().mark_bar(width=30).encode(y='q1:Q', y2='q3:Q'),
    alt.Chart().mark_tick(color='white', width=30).encode(y='median:Q'),
).properties(
    title='Distribution of decile_score by sex',
    width=150
)

decile_by_race = alt.LayerChart(data=df_compare_race).transform_aggregate(
    min="min(decile_score)",
    max="max(decile_score)",
    mean="mean(decile_score)",
    q1="q1(decile_score)",
    median="median(decile_score)",
    q3="q3(decile_score)",
    groupby=['race']
).encode(
    x='race:N',
    tooltip=['min:Q', 'q1:Q', 'median:Q','mean:Q', 'q3:Q', 'max:Q']
).add_layers(
    alt.Chart().mark_rule().encode(y='min:Q', y2='max:Q'),
    alt.Chart().mark_bar(width=30).encode(y='q1:Q', y2='q3:Q'),
    alt.Chart().mark_tick(color='white', width=30).encode(y='median:Q'),
).properties(
    title='Distribution of decile_score by race',
    width=150
)
alt.hconcat(decile_by_sex, decile_by_race)

In [ ]:
recid_race_chart=alt.Chart(df_compare_race, height = 200, width = 300).mark_bar().encode(
    alt.X('recid_within_timeframe:N'),
    y='count()',
    column='race:N',
).properties(
    title='Count of recid by race'
)
recid_sex_chart=alt.Chart(df3, height = 200, width = 300).mark_bar().encode(
    alt.X('recid_within_timeframe:N'),
    y='count()',
    column='sex:N',
).properties(
    title='Count of recid by sex'
)
alt.vconcat(recid_race_chart, recid_sex_chart)

In [ ]:
# Tras observar la relación de estas variables con el "decile_score" y si han reincidido o no. A continuación veremos como influye la edad en la reincidencia
# y en el decile score. Esto lo veremos con el dataframe df que no conteine información de raza aislada.
desitribution_recid_age=alt.LayerChart(data=df3).transform_aggregate(
    min="min(age)",
    max="max(age)",
    mean="mean(age)",
    q1="q1(age)",
    median="median(age)",
    q3="q3(age)",
    groupby=['recid_within_timeframe']
).encode(
    x='recid_within_timeframe:N',
    tooltip=['min:Q', 'q1:Q', 'median:Q','mean:Q', 'q3:Q', 'max:Q']
).add_layers(
    alt.Chart().mark_rule().encode(y='min:Q', y2='max:Q'),
    alt.Chart().mark_bar(width=30).encode(y='q1:Q', y2='q3:Q'),
    alt.Chart().mark_tick(color='white', width=30).encode(y='median:Q'),
).properties(
    title='disitribution of age by recid',
    width=150
)
age_by_sex=alt.LayerChart(data=df3).transform_aggregate(
    min="min(age)",
    max="max(age)",
    mean="mean(age)",
    q1="q1(age)",
    median="median(age)",
    q3="q3(age)",
    groupby=['sex']
).encode(
    x='sex:N',
    tooltip=['min:Q', 'q1:Q', 'median:Q','mean:Q', 'q3:Q', 'max:Q']
).add_layers(
    alt.Chart().mark_rule().encode(y='min:Q', y2='max:Q'),
    alt.Chart().mark_bar(width=30).encode(y='q1:Q', y2='q3:Q'),
    alt.Chart().mark_tick(color='white', width=30).encode(y='median:Q'),
).properties(
    title='Age by sex',
    width=150
)
age_by_race=alt.LayerChart(data=df_compare_race).transform_aggregate(
    min="min(age)",
    max="max(age)",
    mean="mean(age)",
    q1="q1(age)",
    median="median(age)",
    q3="q3(age)",
    groupby=['race']
).encode(
    x='race:N',
    tooltip=['min:Q', 'q1:Q', 'median:Q','mean:Q', 'q3:Q', 'max:Q']
).add_layers(
    alt.Chart().mark_rule().encode(y='min:Q', y2='max:Q'),
    alt.Chart().mark_bar(width=30).encode(y='q1:Q', y2='q3:Q'),
    alt.Chart().mark_tick(color='white', width=30).encode(y='median:Q'),
).properties(
    title='Age by race',
    width=150
)
alt.hconcat(desitribution_recid_age, age_by_sex,age_by_race)

In [ ]:
# También es interesante viualizar el tipo de crimen cometido a la hora de registrar a la persona en el sistema, donde F es Felony y M es misdemeanor. 
# Es decir, F es más grave que M.
charge_type_recid= alt.Chart(df3, height = 200, width = 300).mark_bar().encode(
    alt.X('c_charge_degree:N'),
    y='count()',
    column='recid_within_timeframe:N',
).properties(
    title='Counts of Charge type'
)

charge_type_race=alt.Chart(df_compare_race, height = 200, width = 300).mark_bar().encode(
    alt.X('c_charge_degree:N'),
    y='count()',
    column='race:N',
).properties(
    title='Charge type by race'
)
charge_type_sex=alt.Chart(df3, height = 200, width = 300).mark_bar().encode(
    alt.X('c_charge_degree:N'),
    y='count()',
    column='sex:N',
).properties(
    title='Counts of Charge type'
)

In [ ]:
# Se muestran por separado ya que al intentar concatenar las gráficas nos devolvia un error.
charge_type_recid

In [ ]:
charge_type_race

In [ ]:
charge_type_sex

Se puede concluir de estas visualizaciones que una mayor decile score para los colectivos 'Hombre' y 'raza afroamericana' puede estar justificada por dos razones:
La primera, vemos que las reincidencias se producen a edades más tempranas de una manera generalizada. Se ha visto que el porcentaje de casos de personas de raza afroamericana de menor edad es más alta en comparación con aquellos de raza caucásica.
La segunda, un mayor procentaje de casos de clase F (felony) reinciden, y la proporción de estos casos respeto a aquellos de calse M (misdemeanor) es mayor en los subgrupos de 'hombres' y 'raza afroamericana'.

# ¿Para qué tipo de riesgos, el de delitos generales o el de delitos violentos, tiene el sistema más capacidad predictiva?

In [ ]:
prediccion_generales=df3.groupby('decile_score')['recid_within_timeframe'].apply(lambda x: (x == 'YES').mean())\
.rename('porcentaje_reincidencias')\
.reset_index()

general_chart = alt.Chart(prediccion_generales, title = 'Porcentaje de reincidencias por decile score').mark_bar()\
                        .encode( x = alt.X('porcentaje_reincidencias:Q', axis=alt.Axis(format='.0%'), title='Porcentaje de reincidencias en un mismo periodo de tiempo'), y = alt.Y('decile_score:N', title = 'Decile Score'))\
                        .resolve_scale(y = 'independent')

prediccion_violentos=df3.groupby('v_decile_score')['violent_recid_within_timeframe'].apply(lambda x: (x == 'YES').mean())\
.rename('porcentaje_reincidencias_violentas')\
.reset_index()

violent_chart=alt.Chart(prediccion_violentos, title = 'Porcentaje de reincidencias violentas por v_decile score').mark_bar()\
                        .encode( x = alt.X('porcentaje_reincidencias_violentas:Q', axis=alt.Axis(format='.0%'),title='Porcentaje de reincidencias violentas en un mismo periodo de tiempo'), y = alt.Y('v_decile_score:N', title = 'Violent decile Score'))\
                        .resolve_scale(y = 'independent')
alt.hconcat(general_chart,violent_chart)

Se puede ver por los porcentajes de las dos gráficas que la capacidad predicitva de los delitos generales es mucho mayor que el de los delitos violentos. Dicho esto, se desconoce cual es el periodo de tiempo contemplado para las probilidades asignadas en las columnas decile_score y v_decile_score y tampoco los porcentajes de reincidencia esperadas relativos a cada puntuación. Dado que esto se desconoce no se puede hacer una valoración sobre cómo de apropiados son los valores de decile_score y v_decile_score.